In [2]:
import os
import pandas as pd
import networkx as nx
from collections import defaultdict, Counter
import random
import numpy as np
import json
from data_loader.data_loaders import DataLoader
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

In [3]:
metrics = ['loss', 'accuracy', 'precision', 'recall', 'roc_auc',
     'pr_auc', 'f1_score', 'mcc']

def get_comparison_bars(runs, metrics, title):
    results_file = lambda run: f'saved/log/DrugCombDB/{run}/test_results.csv'

    df1 = pd.read_csv(results_file(runs[0]['path']))
    df2 = pd.read_csv(results_file(runs[1]['path']))

    fig = go.Figure(data=[
        go.Bar(name=runs[0]['name'], x=metrics, y=df1.mean()[metrics], error_y=dict(type='data', array=df1.std()[metrics])),
        go.Bar(name=runs[1]['name'], x=metrics, y=df2.mean()[metrics], error_y=dict(type='data', array=df2.std()[metrics]))
    ])
    fig.update_layout(barmode='group', title=title)
    fig.show()

Comparison between using graph and going graphless

In [31]:
runs = {
    0: {
        'path': '1206_182941',
        'name': "Uses Graph"
    },
    1: {
        'path': '1206_183245',
        'name': "Graphless"
    }
}
get_comparison_bars(runs, metrics, "Impact of PPI Graph on Performance")

Comparison between cleanup and no cleanup

In [29]:
runs = {
    0: {
        'path': '1206_182941',
        'name': "cleaned up"
    },
    1: {
        'path': '1206_185325',
        'name': "not cleaned up"
    }
}
get_comparison_bars(runs, metrics, "Impact of Neighborhood Cleaning on Performance")

In [4]:
runs = {
    0: {
        'path': '1206_182941',
        'name': "Uses Graph"
    },
    1: {
        'path': '1207_004035',
        'name': "Graphless"
    }
}
get_comparison_bars(runs, metrics, "Impact of PPI Graph on Performance (different parameters)")